In [ ]:
import os
import json

import numpy as np

from pymedphys.level1.msqconnect import mosaiq_connect
from pymedphys.level1.configutilities import (
    get_cache_filepaths, get_mu_density_parameters,
    get_index, get_centre, get_sql_servers, get_sql_servers_list,
    get_filepath
)
from pymedphys.level2.msqdelivery import multi_fetch_and_verify_mosaiq
from pymedphys.level3.analyselogfiles import (
    analyse_single_hash, find_consecutive_logfiles, get_field_id_key_map
)

In [ ]:
with open('config.json') as config_file:
    config = json.load(config_file)

In [ ]:
index = get_index(config)
file_hashes = np.array(list(index.keys()))

field_types = np.array([
    index[file_hash]['delivery_details']['field_type']
    for file_hash in file_hashes
])

file_hashes = file_hashes[field_types == 'DMLC']

is_qa = np.array([
    index[file_hash]['delivery_details']['qa_mode']
    for file_hash in file_hashes
])

file_hashes = file_hashes[np.invert(is_qa)]

machine = np.array([
    index[file_hash]['logfile_header']['machine']
    for file_hash in file_hashes
])

# limit to RCCC for now
file_hashes = file_hashes[(machine == '2619') | (machine == '2694')]
np.random.shuffle(file_hashes)

# might need course information
index[file_hashes[0]]

In [ ]:
patient_grouped_fields = dict()
field_id_grouped_hashes = dict()

for file_hash in file_hashes:
    delivery_details = index[file_hash]['delivery_details']
    patient_id = delivery_details['patient_id']
    field_id = delivery_details['field_id']
    
    if patient_id not in patient_grouped_fields:
        patient_grouped_fields[patient_id] = set()
        
    patient_grouped_fields[patient_id].add(field_id)
    
    if field_id not in field_id_grouped_hashes:
        field_id_grouped_hashes[field_id] = []
        
    field_id_grouped_hashes[field_id].append(file_hash)

In [ ]:
patient_ids = list(patient_grouped_fields.keys())

In [ ]:
patient_id = '002911'
fields = patient_grouped_fields[patient_id]

In [ ]:
for field in fields:
    print(field_id_grouped_hashes[field])

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    for field in fields:
        mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(
            cursor, field)
        print(set(mosaiq_delivery_data.gantry))

In [ ]:
# field_id_key_map = get_field_id_key_map(index)

In [ ]:
# keys.append(keys[0])

In [ ]:
def group_consecutive_logfiles(file_hashes, index):
    times = np.array([
        index[key]['local_time']
        for key in file_hashes
    ]).astype(np.datetime64)

    sort_reference = np.argsort(times)
    file_hashes = file_hashes[sort_reference]
    times = times[sort_reference]

    hours_4 = np.array(60 * 60 * 4).astype(np.timedelta64)
    split_locations = np.where(np.diff(times) >= hours_4)[0] + 1

    return np.split(file_hashes, split_locations)


field_id = list(fields)[0]
keys = np.array(field_id_grouped_hashes[field_id])
keys = np.hstack([keys, np.array(keys[1])])  # Spin this out into a unit test

group_consecutive_logfiles(keys, index)

In [ ]:


# field_id = list(fields)[0]
# keys = np.array(field_id_grouped_hashes[field_id])
# keys = np.hstack([keys, np.array(keys[1])])

# times = np.array([
#     index[key]['local_time']
#     for key in keys
# ]).astype(np.datetime64)

# sort_reference = np.argsort(times)
# keys = keys[sort_reference]
# times = times[sort_reference]

# hours_4 = np.array(60 * 60 * 4).astype(np.timedelta64)
# split_locations = np.where(np.diff(times) >= hours_4)[0] + 1

# np.split(times, split_locations)

# # split_locations

In [ ]:
# np.split(keys, split_locations)

In [ ]:
times

In [ ]:
hours_4

In [ ]:

file_hash = field_id_grouped_hashes[field_id][0]

find_consecutive_logfiles(field_id_grouped_hashes, field_id, file_hash, index, config)

In [ ]:
# patient_grouped_fields

In [ ]:
# field_id_grouped_hashes

In [ ]:
get_filepath(index, config, file_hash)

In [ ]:
# patient_grouped_hashes

In [ ]:
# patient_ids = [
#     index[file_hash]['delivery_details']['patient_id']
#     for file_hash in file_hashes
# ]

# # patient_ids

In [ ]:
len(file_hashes)

In [ ]:
# filepaths = np.array([
#     os.path.join(
#         config['linac_logfile_data_directory'],
#         'indexed',
#         index[file_hash]['filepath'])
#     for file_hash in file_hashes
# ])
# filepaths

In [ ]:
file_hash = file_hashes[0]

logfile_index_record = index[file_hash]
field_id = logfile_index_record['delivery_details']['field_id']

In [ ]:
with mosaiq_connect('msqsql') as cursor:
    mosaiq_delivery_data = multi_fetch_and_verify_mosaiq(
        cursor, field_id)

In [ ]:
mosaiq_delivery_data.gantry